In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.ar_model import AutoReg

plt.rcParams["figure.dpi"] = 100

def rms(s: pd.Series) -> float:
    return np.sqrt((s**2).mean())

# Load data

In [ ]:
from sensor import create_raw_data_file
create_raw_data_file()

In [ ]:
# Read all data from parquet file
data = pd.read_parquet("raw_data_all.parquet")

# For simplicity, select sensor 3
data = data[data["sensor"] == "node_03"]

# Replace 0-measurements with missing
data.loc[data["Leq"] == 0, "Leq"] = None

# For simplicity, downsample to 10 minutes
data = data.resample("1min").median()

# Forward-fill missing values
data = data.fillna(method="ffill")

# Add some extra columns
data["hour"] = data.index.hour
data["dow"] = data.index.dayofweek
data["workday"] = (data.index.dayofweek < 5).astype(int)
data["doy"] = data.index.dayofyear
data["week"] = data.index.week
data["workhour"] = data["hour"].isin(range(6,21))*data["hour"]

data.head()

In [ ]:
fig = px.line(data, y="Leq", title=f"Raw data resampled to {data.index.freq.n} minutes", color="week")
fig.show()

In [ ]:
decomposed = sm.tsa.seasonal_decompose(data["Leq"], period=pd.Timedelta("24hours") // data.index.freq)
fig = decomposed.plot()
fig.set_size_inches(10,10)

In [ ]:
decomposed = sm.tsa.seasonal_decompose(data["Leq"], period=pd.Timedelta("1W") // data.index.freq)
fig = decomposed.plot()
fig.set_size_inches(10,10)

In [ ]:
# Split in training and test
train = data[data["week"].isin([7, 8, 9, 10, 11])]
test = data[data["week"].isin([12, 13, 14, 15])]
train_test = pd.concat([train, test])
train_test.loc[train.index, "dataset"] = "train"
train_test.loc[test.index, "dataset"] = "test"

# Linear model

In [ ]:
model_formula = "C(dow) + C(hour):C(dow)" # week"
# model_formula = "C(workday) + C(workhour):C(workday)"

linmodel = smf.ols(formula=f"Leq ~ {model_formula}", data=train).fit()
linmodel_resid = train_test["Leq"] - linmodel.predict(train_test)
linmodel.summary()

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=train.index, y=train["Leq"], name="Train"), row=1, col=1)
fig.add_trace(go.Scatter(x=test.index, y=test["Leq"], name="Test"), row=1, col=1)
fig.add_trace(go.Scatter(x=train_test.index, y=linmodel.predict(train_test), name="Model"), row=1, col=1)

fig.add_trace(go.Scatter(x=train_test.index, y=linmodel_resid, name="Residual"), row=2, col=1)

fig.update_layout(
    title="Linear model results",
    width=1000,
    height=800,
    hovermode="x"
)

In [ ]:
linmodel_resid.groupby(train_test["dataset"]).apply(rms)

# ARX model

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(10, 10))
fig = sm.graphics.tsa.plot_acf(linmodel.resid, lags=30, ax=ax[0])
fig = sm.graphics.tsa.plot_pacf(linmodel.resid, lags=30, ax=ax[1])

In [ ]:
exog_train = patsy.dmatrix(model_formula, train)
exog_test = patsy.dmatrix(model_formula, test)

In [ ]:
lags = math.ceil(pd.Timedelta("4min") / train.index.freq)
lags

In [ ]:
arxmodel = AutoReg(endog=train["Leq"], lags=lags, exog=exog_train).fit()
arxmodel_pred = pd.concat([
    arxmodel.predict(),
    arxmodel.predict(start=test.index[0], end=test.index[-1], exog_oos=exog_test)
])
arxmodel_resid = train_test["Leq"] - arxmodel_pred

In [ ]:
fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=train.index, y=train["Leq"], name="Train"), row=1, col=1)
fig.add_trace(go.Scatter(x=test.index, y=test["Leq"], name="Test"), row=1, col=1)
fig.add_trace(go.Scatter(x=train_test.index, y=arxmodel_pred, name="Model"), row=1, col=1)

fig.add_trace(go.Scatter(x=train_test.index, y=arxmodel_resid, name="Residual"), row=2, col=1)

fig.update_layout(
    title="ARX model results",
    width=1000,
    height=800,
    hovermode="x"
)

In [ ]:
arxmodel_resid.groupby(train_test["dataset"]).apply(rms)

# Model comparison

In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=train_test.index, y=train_test["Leq"], name="Measured"), row=1, col=1)
fig.add_trace(go.Scatter(x=train_test.index, y=linmodel.predict(train_test), name="LinModel"), row=1, col=1)
fig.add_trace(go.Scatter(x=train_test.index, y=arxmodel_pred, name="ARXModel"), row=1, col=1)
fig.update_layout(
    title="Model comparison",
    hovermode="x"
)

# ARX with dynamic forecasting

In [ ]:
exog = patsy.dmatrix(model_formula, train_test)
model = AutoReg(endog=train_test["Leq"], lags=lags, exog=exog).fit()

In [ ]:
forecast_period = pd.Timedelta("3hour")

t = train_test.index[0] + forecast_period
preds = pd.Series(dtype=float)
while t < train_test.index[-1] - forecast_period:
    preds = preds.append(model.predict(start=t, end=t + forecast_period, dynamic=t))
    t += forecast_period

In [ ]:
fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
fig.add_trace(go.Scatter(x=train_test.index, y=train_test["Leq"], name="Measured"), row=1, col=1)
fig.add_trace(go.Scatter(x=preds.index, y=preds, name="ARX Forecast"), row=1, col=1)
fig.update_layout(
    title="Dynamic ARX forecasting",
    hovermode="x"
)

In [ ]:
resid = train_test["Leq"] - preds
rms(resid)